<a href="https://colab.research.google.com/github/ramalias/urban-expansion/blob/main/main-urban-expansion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#install library if needed

!apt-get update -qq
!apt-get install -y gdal-bin python3-gdal > /dev/null
!pip install rasterio geopandas shapely pyproj osmnx==1.9.2

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 110.7 MB/s eta 0:00:00


In [7]:
#library
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import os, glob, subprocess #system and file handling
from rasterio.windows import from_bounds #create window object
import matplotlib.pyplot as plt #visualization
import numpy as np
import osmnx as ox #open street map
from pyproj import Transformer #coordinate transformer
import geopandas as gpd
import shapely
from shapely.geometry import Polygon
from rasterio import features
import rasterio  #raster data handling
from matplotlib.patches import Patch

In [4]:
#create main project folder
#run one time only

project_root = '/content/drive/MyDrive/project/urban-exansion'
os.makedirs(project_root, exist_ok=True)

#create folder inside project folder
os.makedirs(os.path.join(project_root, 'data'), exist_ok=True)

print('Workspace created at:', project_root)

Workspace created at: /content/drive/MyDrive/project/urban-exansion


In [ ]:
#define sentinel folder

sentinel_dir="/content/drive/MyDrive/project/urban-exansion/data/01_Dataset_sentinel"